# Crew to Tailor Job Applications

install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume_praise.md')
# perform RAG over fake resume (semantic search)
semantic_search_resume = MDXSearchTool(mdx='./fake_resume_praise.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.02it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [5]:
# Agent 1: Researcher
# tech job researcher - understand needs behind job posting
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [6]:
# Agent 2: Profiler
# Research on job applicant and identify what stands out for the job
# highlight strong points
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [12]:
# Agent 3: Resume Strategist
# update the resume 
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [7]:
# Agent 4: Interview Preparer
# send key points for interviews/talking points
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [8]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [10]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, Portfolio ({portfolio_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
# tailor resume for job posting
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
# write interview materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [25]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/matchgroup/c69a83e8-d2c5-46d4-a544-45a984370666',
    'github_url': 'https://github.com/thetosy',
    'portfolio_url': 'https://www.praiseolukilede.com',
    'personal_writeup' : """
Praise is a skilled professional with experience as a Research Assistant at the University of Southern California,
a Software DevOps Engineer at Leslie's, and a Software Engineer at Boterna. His responsibilities have included leveraging data analysis and 
machine learning techniques, conducting data preprocessing and analysis, and enhancing system performance through design and construction.
He has hands-on experience in Docker, Kubernetes, and KubeFlow for developing scalable machine learning applications and 
in various programming languages like Python, Java, and SQL, along with specializations in Deep Learning, Generative AI, MLOps, and Natural Language Processing.
"""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [26]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/matchgroup/c69a83e8-d2c5-46d4-a544-45a984370666) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/thetosy) URLs, Portfolio (https://www.praiseolukilede.com) URLs, and personal write-up (
Praise is a skilled professional with experience as a Research Assistant at the University of Southern California,
a Software DevOps Engineer at Leslie's, and a Software Engineer at Boterna. His responsibilities have included leveraging data analysis and 
machine learning techniques, conducting data preprocessing and analysis, and enhancing system perfor

 

thetosy · GitHub
Skip to content
Navigation Menu
Toggle navigation
 Sign in
 Product
Actions
 Automate any workflow
Packages
 Host and manage packages
Security
 Find and fix vulnerabilities
Codespaces
 Instant dev environments
Copilot
 Write better code with AI
Code review
 Manage code changes
Issues
 Plan and track work
Discussions
 Collaborate outside of code
Explore
 All features
 Documentation
 GitHub Skills
 Blog
 Solutions
For
 Enterprise
 Teams
 Startups
 Education
By Solution
 CI/CD & Automation
 DevOps
 DevSecOps
Resources
 Learning Pathways
 White papers, Ebooks, Webinars
 Customer Stories
 Partners
 Open Source
GitHub Sponsors
 Fund open source developers
The ReadME Project
 GitHub community articles
Repositories
 Topics
 Trending
 Collections
Pricing
Search or jump to...
Search code, repositories, users, issues, pull requests...
 Search
Clear
 Search syntax tips
 Provide feedback
We read every piece of feedback, and take your input very seriously.
Include my email addres

Final Answer:
# Praise Olukilede
- Email: praise.olukilede@example.dev
- Phone: +1 11 111 11111

## Profile
Praise is a skilled professional with experience as a Research Assistant at the University of Southern California, Software DevOPs Engineer at Leslie's, and Software Engineer at Boterna. His responsibilities have included leveraging data analysis and machine learning techniques, conducting data preprocessing and analysis, and enhancing system performance through design and construction. He has hands-on experience in Docker, Kubernetes and, KubeFlow for developing scalable machine learning applications and in various programming languages like Python, Java, and SQL, along with specializations in Generative AI, MLOps, and Natural Language Processing.

## Projects
1.    Emotional State Classification:  Authored a paper on the impact of maternal emotional state on mother-infant interactions, analyzing 235 videos spanning 13 hours. Developed a multimodal deep learning model using faci

I need to gather relevant information from Praise Olukilede's resume and create interview questions and talking points based on the necessary skills, qualifications, experiences, and projects listed.

Action: Read a file's content
Action Input: {} 


# Praise Olukilede
- Email: praise.olukilede@example.dev
- Phone: +1 11 111 11111

## Profile
Praise is a skilled professional with experience as a Research Assistant at the University of Southern California, Software DevOPs Engineer at Leslie's, and Software Engineer at Boterna. His responsibilities have included leveraging data analysis and machine learning techniques, conducting data preprocessing and analysis, and enhancing system performance through design and construction. He has hands-on experience in Docker, Kubernetes and, KubeFlow for developing scalable machine learning applications and in various programming languages like Python, Java, and SQL, along with specializations in Generative AI, MLOps, and Natural Language Processing

- Dislplay the generated `tailored_resume.md` file.

In [27]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Praise Olukilede
- Email: praise.olukilede@example.dev
- Phone: +1 11 111 11111

## Profile
Praise is a skilled professional with experience as a Research Assistant at the University of Southern California, Software DevOPs Engineer at Leslie's, and Software Engineer at Boterna. His responsibilities have included leveraging data analysis and machine learning techniques, conducting data preprocessing and analysis, and enhancing system performance through design and construction. He has hands-on experience in Docker, Kubernetes and, KubeFlow for developing scalable machine learning applications and in various programming languages like Python, Java, and SQL, along with specializations in Generative AI, MLOps, and Natural Language Processing.

## Projects
1.    Emotional State Classification:  Authored a paper on the impact of maternal emotional state on mother-infant interactions, analyzing 235 videos spanning 13 hours. Developed a multimodal deep learning model using facial and audio features, achieving 96% accuracy in predicting maternal depression 
2.    Deep Learning for Image Compression:  Utilized a stable diffusion model for image compression, resulting in a 9% efficiency gain and an 8% improvement in PSNR while maintaining SSIM integrity. Additionally, leveraging the Space-Time U-Net technique led to an impressive 800x compression ratio in videos 
3.    Automated Audio Insights Pipeline:  Utilized Amazon Bedrock to architect an event-driven pipeline for providing insights on audio recordings of two speakers, with automation facilitated by AWS Lambda. Initiated the workflow with Amazon Transcribe for audio-to-text conversion and diarization, followed by processing the transcribed text, with prompt engineering, through an LLM, using Amazon's Titan framework, for insight generation
4.    On-time Flight Tracking:  Developed and deployed an airline on-time arrival prediction model using the mirrored distributed training strategy for faster learning, achieving 81% accuracy. Utilized GCP Vertex AI for both online and batch inference deployments. Also, incorporated a Sampled-Shapley method-based eXplainable AI (XAI) model to offer prediction insights 
5.    RAG Chatbot:  Leveraging the Retrieval Augmented Generation (RAG) Workflow, I developed an enhanced chatbot proficient in accurately answering questions by accessing information in its vector store. Enforced diversity and relevance in search results using Maximal Marginal Relevance (MMR). Utilized LangChain framework alongside OpenAI GPT-3.5-turbo as the Large Language Model (LLM)
6.    RLHF FineTuning:  Fine-tuned the FLAN-T5 model via RLHF (Reinforcement Learning with Human Feedback) to produce less-toxic summaries. Integrated the proximal policy optimization (PPO) RL algorithm and the LoRA reparameterization method for efficient parameter fine-tuning. Incorporated META AI's hate speech model as the reward mechanism and utilized the DialogSum Dataset from HF🤗 for training. Achieved a mean toxicity score improvement of 53.88%

## Work History

### University of Southern California: Research Assistant - 2023 - 2024
- Tech Stack: [python, SQL, scikit-learn, TF, pandas, NumPy, Matplotlib, Git]
- Praise provided crucial insights that informed policy and decision making by leveraging data collection, analysis and machine learning techniques to store, preprocess, analyze, and interpret economic time series data
- Praise contributed to shaping strategies that enhance socioeconomic health of multiple transportation sectors 

### Leslie's: Software Development Operations Engineer — 2021 - 2022
- Tech Stack: [python, SQL, K8s, Docker, GCP, Dynatrace, scikit-learn, pandas, NumPy, Matplotlib, pytest, Git]
-  Monitored and executed queries on the production server/database, conducting data preprocessing, analysis and visualizations, and model development to facilitate informed decision-making
-  Developed and implemented daily reports to evaluate system health, ensuring peak performance at all times
-  Designed and automated test suites to optimize testing workflows and boost overall efficiency
-  Conducted Root Cause Analysis (RCA) for unexpected performance, resulting in a 70% improvement in resolution time

### Boterna: Software Engineer — 2020 - 2021
- Tech Stack: [python, C++, XCTU, SQL, Matplotlib, Beebotte, MQTT, Arduino, Git]
- Praise enhanced performance of a wireless sensor network by designing and constructing it to monitor diverse system parameters
- He programmed microcontrollers for monitoring operations, collected data, and uploaded it to a cloud database
- He Conducted detailed analysis and generated visualizations for comprehensive insights

## Education

### M.Sc in Computer Science
University of Southern California - 3.78 GPA

### Machine Learning for Production
Coursera (DeepLearning.AI) - Certification

### Natural Language Processing Specialization
Coursera (DeepLearning.AI) - Certification

### Generative AI With Large Language Models
Coursera (DeepLearning.AI) - Certification

### MLExpert
AlgoExpert - Certification

### B.Sc. in Computer Engineering
Oral Roberts University - 3.66 GPA

## Technical Skills
Programming: Python, Java, C/C++, SQL, Bash, Matlab; DBMS: MySQL, No-SQL, ChromaDB, Neo4j
Frameworks: Apache Beam, Snorkel, TensorFlow Extended (TFX), TFServing, TensorFlow, PyTorch, PyTorch Lightning, Hugging Face, FastAPI, pytest, PySpark, LangChain, Unstructured, crewAI, Istio
Libraries: Librosa, NumPy, Pandas, SciPy, scikit-learn, Keras, KerasTuner, NLTK, Boto3, SentencePiece, MultiBench, ML Metadata, Matplotlib, Seaborn, PyG, PEFT, TRL, Diffusers, Quanto, OpenAI, SHAP, Pydantic, XGBoost, Locust, Deepgram, kfp
Tools: Git, GitHub Actions, Dynatrace, Anthos Service Mesh, Kubernetes, Kubeflow, Docker, Weights and Biases
Cloud Services: AWS (S3, Lambda, Bedrock, SageMaker), GCP (Vertex AI, BigQuery, Kubernetes Engine, DLP), Snowflake
Technical Expertise: Machine Learning/AI: Classification, Regression, Clustering, Anomaly Detection, Time Series Analysis and Forecasting, Feature Extraction, HMM, RL, Contrastive Learning; NLP: Transformers, Named Entity Recognition, Neural Machine Translation, Language Models, Text/Speech Processing, Automatic Speech Recognition, Speaker Recognition; Deep Learning: Network Architectures, Diffusion Models (U-Net), GNN, Learning Techniques, Regularization, Optimizers, Adaptation, Transfer Learning, Loss Functions, Multi-Modal Learning; Data Eng./MLOps: Data Augmentation, REST APIs, Active Learning, A/B Testing, Canary Deployment, HyperParameter Tuning and Optimization, Pruning, Quantization, Dimensionality Reduction, ETL Pipeline Development, Distributed Training, Distributed load testing, Knowledge Distillation, Knowledge Graphs
Soft Skills: Problem Solving, Communication, Presentation, Team Builder, Analytical Thinking

- Dislplay the generated `interview_materials.md` file.

In [28]:
display(Markdown("./interview_materials.md"))

Based on Praise Olukilede's resume and qualifications, here are some potential interview questions and talking points for the candidate:

1. Can you walk us through a project where you utilized statistical modeling and causal inference techniques to derive meaningful insights?
2. How have you applied your knowledge of experimentation in past projects to drive decision-making and improve outcomes?
3. Could you discuss a scenario where you used Python/R, Tableau, or Mode to analyze data and present findings to stakeholders?
4. What experience do you have with SQL and how have you used it in data analysis or system performance enhancement?
5. Can you provide an example of a project where you implemented machine learning algorithms for data preprocessing and analysis?
6. How have you leveraged Docker, Kubernetes, and KubeFlow in developing scalable machine learning applications in your previous roles?
7. Describe a time when you specialized in Deep Learning, Generative AI, MLOps, or Natural Language Processing and the impact it had on the project.
8. Share a challenging situation you encountered in a project and how you overcame it using your technical skills and expertise.
9. How do you ensure effective communication with team members while working on technical projects? Can you provide examples of successful collaborations?
10. What interests you the most about data analysis, machine learning, system performance enhancement, and the specialized areas you have experience in?
11. How do you approach presenting complex technical concepts in a clear and concise manner, especially in a team setting or to stakeholders?
12. Discuss a project where you contributed significantly to enhancing system performance through design and construction and the results achieved.
13. Can you explain a time when you had to conduct data preprocessing and analysis to derive actionable insights for decision-making?
14. Share details about a project where you developed scalable machine learning applications and the technologies or frameworks you utilized.
15. How do you stay updated with the latest trends and advancements in the field of data science and analytics, especially in areas like Deep Learning and MLOps?

These questions and talking points aim to help the candidate showcase their skills, experiences, and qualifications effectively during the interview process.